In [279]:
!pip install tensorflow==2.9.1 tensorflow_datasets==4.6.0

from __future__ import absolute_import, division, print_function, unicode_literals

import sys

import os
import re
import numpy as np
#from time import time
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import pandas as pd
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tensorflow.keras.preprocessing.sequence import pad_sequences
import math

tf.keras.utils.set_random_seed(1234)

print(f"Tensorflow version {tf.__version__}")

Tensorflow version 2.9.1


In [280]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Running on TPU {}".format(tpu.cluster_spec().as_dict()["worker"]))
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print(f"REPLICAS: {strategy.num_replicas_in_sync}")

REPLICAS: 1


In [281]:
!pip install tokenizers


In [282]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/'BERIL_PROJECT'/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/BERIL_PROJECT


In [283]:
%ls


BIG_DATASET.csv  CODE.ipynb  GENERATING_DATASET.ipynb  SHORT_DATASET.csv


In [284]:
df = pd.read_csv('SHORT_DATASET.csv')

In [285]:
len(df)

19

In [286]:
primera_columna = df.iloc[:, 0]
segunda_columna = df.iloc[:, 1]

https://www.youtube.com/watch?v=MlDP2BVWjS0

https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0

In [287]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # removing contractions  didn t
    sentence = re.sub(r"i'm", "i am", sentence)
    sentence = re.sub(r"he's", "he is", sentence)
    sentence = re.sub(r"she's", "she is", sentence)
    sentence = re.sub(r"it's", "it is", sentence)
    sentence = re.sub(r"that's", "that is", sentence)
    sentence = re.sub(r"what's", "that is", sentence)
    sentence = re.sub(r"where's", "where is", sentence)
    sentence = re.sub(r"how's", "how is", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"didn t", "did not", sentence)
    sentence = re.sub(r"can't", "cannot", sentence)
    sentence = re.sub(r"n't", " not", sentence)
    sentence = re.sub(r"n'", "ng", sentence)
    sentence = re.sub(r"'bout", "about", sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [288]:
print(primera_columna)

0     I am thinking of soon upgrading from my now al...
1     I am thinking of soon upgrading from my now al...
2     I am thinking of soon upgrading from my now al...
3     I am thinking of soon upgrading from my now al...
4     I am thinking of soon upgrading from my now al...
5     I have a Poco M3 Pro 5G and it used to work fi...
6     I have a Poco M3 Pro 5G and it used to work fi...
7     Redmi note 12 pro 5G can be charged at 67 Watt...
8     Redmi note 12 pro 5G can be charged at 67 Watt...
9     Redmi note 12 pro 5G can be charged at 67 Watt...
10                              Which one do you prefer
11    Im starting to think that xiaomi might have fo...
12    Im starting to think that xiaomi might have fo...
13    Im starting to think that xiaomi might have fo...
14    Im starting to think that xiaomi might have fo...
15    Im starting to think that xiaomi might have fo...
16    There seems to have a massive transparent scre...
17    Last week I checked on updates and found t

In [289]:
primera_columna = primera_columna.apply(preprocess_sentence)

In [290]:
print(segunda_columna)

0     Hey, I am from Germany, and have had both .. F...
1     If you only care about performance, get the X6...
2     Poco X6 Pro is a rebranded Redmi K70E*, it's c...
3     13T Pro here just received the update. Didn't ...
4                                                X6 Pro
5     Well, either it's a software issues of the PMI...
6     i would let the battery completly die out for ...
7     No, because Xiaomi phones use Mi-FC standard f...
8     the charger is designed by the manufacturer to...
9     How do people even come up weiHow do people co...
10    YouTube forces an equalizer onto the users, wh...
11    See Xiaomi Hyper OS update tracker table [here...
12    Some people using 13t have received hyerOS. Mo...
13        We got one 13T in family, we have the update.
14        Greece here 13t updated a week ago in hyperos
15                     i have mine on indonesian rom ☹️
16                            https://imgur.com/72gT1Ge
17    That happened once to me in an older updat

In [291]:
segunda_columna = segunda_columna.apply(preprocess_sentence)

In [292]:
combined_df = pd.concat([primera_columna, segunda_columna], axis=0)
print(combined_df.shape)
ultimas_columnas = combined_df.tail()
print(ultimas_columnas)

(38,)
14          greece here t updated a week ago in hyperos
15                        i have mine on indonesian rom
16                              https imgur . com gt ge
17    that happened once to me in an older update . ...
18    i do not know , i still have not got the hyper...
dtype: object


In [293]:
tokenizerTotal = Tokenizer(BPE(unk_tolen = "<unk>"))
trainerTotal = BpeTrainer(vocab_size = 2000, special_tokens = ["<s>", "<pad>", "</s>", "<unk>"])
tokenizerTotal.pre_tokenizer = Whitespace()
tokenizerTotal.train_from_iterator(combined_df, trainerTotal)

In [294]:
vueltas = 0
vocab_size = tokenizerTotal.get_vocab_size()
print(vocab_size)
for fila in primera_columna:
  vueltas += 1
  tokenizada = tokenizerTotal.encode(fila)
  print("............................")
  print(fila)

  print(tokenizada.tokens)
  print(tokenizada.ids)
  print("............................")
  if(vueltas == 20):
    break

1029
............................
i am thinking of soon upgrading from my now almost years old remi note pro g uk , global version . i am debating whether to go for the redmi note pro or poco x pro . i have always used my phones only for browsing and fb , but i also watch a lot of k content on yt , so a good display is a must . a semi decent camera would also be nice . i do not feel the camera on my note was particularly good . i also want to maintain the smoothness and quick response of the note . based on all of that , which of the above do you think will be the better choice ? i think they will be roughly the same price here in the uk once the redmi note releases . thanks . edit thanks all for the recommendations . i went with the poco x pro !
['i', 'am', 'thinking', 'of', 'soon', 'upgrading', 'from', 'my', 'now', 'almost', 'years', 'old', 'remi', 'note', 'pro', 'g', 'uk', ',', 'global', 'version', '.', 'i', 'am', 'debating', 'whether', 'to', 'go', 'for', 'the', 'redmi', 'note', 'pr

In [295]:
def tokenizar_y_padding_FINAL(tokenizer, df):
  maximo = 120
  salida = []
  for f in df:
      frase =  [tokenizer.get_vocab_size()] + tokenizer.encode(f).ids + [tokenizer.get_vocab_size() + 1]
      salida.append(frase)
  print("MAXIMO encoder ", maximo)
  print(len(salida))
  salida = pad_sequences(salida, padding="post", maxlen=maximo)
  return salida

In [296]:
questions = tokenizar_y_padding_FINAL(tokenizerTotal, primera_columna)
answers = tokenizar_y_padding_FINAL(tokenizerTotal, segunda_columna)

MAXIMO encoder  120
19
MAXIMO encoder  120
19


In [297]:
def scaled_dot_product(q,k,v, maskLookAhead=None):
  denominador = q.shape[-1]
  parentesis = tf.matmul(q, tf.transpose(k, perm=[0, 1, 3, 2])) / math.sqrt(denominador)
  #maskPadding = tf.cast(tf.equal(parentesis, 0), tf.float32)
  #print("maskPadding!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!", maskPadding)

  if maskLookAhead is not None:
        #maskPadding = tf.maximum(maskPadding, maskLookAhead)
        print("maaaaask look ahead", maskLookAhead)
        #print("maskPadding + look ahead", maskPadding)
        print("PARENTESIS ANTES", parentesis)
        print("MASK * NEG", maskLookAhead * -1e15)
        parentesis += maskLookAhead * -1e15
        print("PARENTESIS DESPUES",parentesis)
        print("---------------------------------------------------------")
  attention = tf.nn.softmax(parentesis, axis=-1)
  print("ATEENTION", attention)
  print("V", v)
  values = tf.matmul(attention, v)
  print("values", values)
  return values

In [298]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
      super(MultiHeadAttention, self).__init__()
      self.d_model = d_model  #es la embedding dimension
      self.num_heads = num_heads
      self.head_dim = d_model // num_heads #cada cabeza tiene esta dimension
      self.qkv_layer = tf.keras.layers.Dense(3 * d_model) #hacemos la dimension de emmbedding por tres para hacer en el mismo tensor q, k y v
      self.linear_layer = tf.keras.layers.Dense(d_model)

  def call(self, inputs):
        x, mask = (
            inputs["x"],
            inputs["mask"]
        )
        print("x", x)
        print("mask", mask)
        #batch_size, sequence_length, d_model = tf.shape(x) #entra como 4 (batch_size), 140 seq_lenght, 512 d_model
        #batch_size, sequence_length, d_model = tf.shape(x).numpy().tolist()
        batch_size, sequence_length, d_model = tf.compat.v1.Session().run(tf.compat.v1.shape(x)).tolist()



        print("batch_size, sequence_length, d_model", batch_size, sequence_length, d_model)
        print("tf.shape(x)", tf.shape(x))
        qkv = self.qkv_layer(x) #lo pasamos de d_model a d_modelx3
        print("tf.shape(qkv) por 3", tf.shape(qkv))
        qkv = tf.reshape(qkv, (batch_size, sequence_length, self.num_heads, 3 * self.head_dim)) #lo dividimos en las cabezas
        print("tf.shape(qkv) le cambiamos a 4 dim", tf.shape(qkv))
        qkv = tf.transpose(qkv, perm=(0, 2, 1, 3))#le cambiamod el orden para que esten las cabezas primero
        print("tf.shape(qkv) cambiamos de orden la cabeza", tf.shape(qkv))
        q, k, v = tf.split(qkv, num_or_size_splits=3, axis=-1)#lo dividimos en 3 por la ultima dimension
        print("qkv", tf.shape(q),tf.shape(k),tf.shape(v))
        values = scaled_dot_product(q, k, v, mask) #lo pasamos por la funcion de atencion que nos devuelve un tensor de 4, 8, 140, 64
        print("values", tf.shape(values))#TENGO QUE METER EN LA LINEA DE ANTES MASKPADDING
        values = tf.reshape(tf.transpose(values, perm=(0, 2, 1, 3)), (batch_size, sequence_length, self.num_heads * self.head_dim)) #lo volvemos a poner primero seqlen y luego la dimension de las cabezas y las juntamos
        print("values cambiad0", tf.shape(values))
        out = self.linear_layer(values)
        print("out", tf.shape(out))
        print("out", out)
        return out



In [299]:
dataset = tf.data.Dataset.from_tensor_slices(
    (
        {"inputs": questions[1:, :], "dec_inputs": answers[1:, :]},
        {"outputs": answers[1:, :]}
    )
    )

In [300]:
print(dataset)
print(dataset.cardinality().numpy())
print(len(dataset))
for fila in dataset.take(1):
    print(fila)
dataset = dataset.cache()
#dataset = dataset.shuffle(len(dataset))
dataset = dataset.batch(2)
#dataset = dataset.prefetch(tf.data.AUTOTUNE)
print(dataset.cardinality().numpy())
for fila in dataset.take(1):
    print(fila)


<TensorSliceDataset element_spec=({'inputs': TensorSpec(shape=(120,), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(120,), dtype=tf.int32, name=None)}, {'outputs': TensorSpec(shape=(120,), dtype=tf.int32, name=None)})>
18
18
({'inputs': <tf.Tensor: shape=(120,), dtype=int32, numpy=
array([  96,  168,  152,   64,  398,   52,  329,    5,  124,   16,  100,
        310,    8,  338,   66,   18,  401,   38,  264,    5,   57,    8,
        159,  275,   58,    8,  224,    6,    8,  374,  243,  185,  139,
        100,   55,  340,    6,   16,  123,   60,  186,   37,  185,   38,
         96,   80,  192,  417,  159,    6,   16,  100,  354,   53,  406,
         37,  402,   52,  386,  410,   66,   37,   80,    6,  393,   38,
        180,   66,  111,    5,  156,   66,   37,  382,  123,   88,  107,
        197,   55,   37,  221,  380,    7,   16,  107,  359,  197,   55,
        407,   37,  177,  274,  211,   36,   37,  201,  265,   37,  160,
         80,  403,    6,  198,    6,  368,  19

In [301]:
dataset

<BatchDataset element_spec=({'inputs': TensorSpec(shape=(None, 120), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(None, 120), dtype=tf.int32, name=None)}, {'outputs': TensorSpec(shape=(None, 120), dtype=tf.int32, name=None)})>

MASKS

In [302]:
### es equivalente a create_look_ahead_mask

def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    return look_ahead_mask

def crear_masks(x, lookAheadMask = None):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    maskA= mask[:, tf.newaxis, tf.newaxis, :]
    maskB = mask[:, tf.newaxis, :,tf.newaxis]
    if lookAheadMask is None:
      seq_len = tf.shape(x)[1]
      lookAheadMask = tf.zeros((seq_len, seq_len))
    maskFinal = tf.maximum(maskA,lookAheadMask)
    maskFinal2 = tf.maximum(maskB,maskFinal)
    return maskFinal2



In [303]:

def create_mask_cross(encoder_inputs, decoder_inputs):
    encMask = tf.cast(tf.math.equal(encoder_inputs, 0), tf.float32)
    decMask = tf.cast(tf.math.equal(decoder_inputs, 0), tf.float32)
    #print("encMask", encMask)
    #print("decMask", decMask)
    encMaskAmp= encMask[:, tf.newaxis, tf.newaxis, :]
    decMaskAmp = decMask[:, tf.newaxis, :,tf.newaxis]
    #print("encMaskAmp", encMaskAmp)
    #print("decMaskAmp", decMaskAmp)
    seq_len = tf.shape(encoder_inputs)[1]
    maskBase = tf.zeros((seq_len, seq_len))
    #print("mask base", maskBase)
    #print("pruebo maximum con encMaskAmp", tf.maximum(encMaskAmp,maskBase))
    #print("pruebo maximum con decMaskAmp", tf.maximum(decMaskAmp,maskBase))
    maskFinal = tf.maximum(encMaskAmp,maskBase)
    maskFinal2 = tf.maximum(decMaskAmp,maskFinal)
    return maskFinal2




EMPEZAMOS CON EL TRANSFORMER

POSITIONAL ENCODER

In [304]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, d_model, max_sequence_length):
        super(PositionalEncoding, self).__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def call(self, inputs):
    #def call(self):
        i2 = tf.range(0, self.d_model, 2, dtype=tf.float32)
        print("i2", i2)
        print("i2 shape", tf.shape(i2))
        denominador = tf.pow(10000.0, i2 / self.d_model)
        print("denominador", denominador)
        print("denominador shape", tf.shape(denominador))
        pos = tf.range(self.max_sequence_length, dtype=tf.float32)
        print("pos", pos)
        print("shape pos", tf.shape(pos))
        posCAMBIADO = tf.reshape(pos, (self.max_sequence_length, 1))
        print("pos CAMBIADO", posCAMBIADO)
        print("shape pos CAMBIADO", tf.shape(posCAMBIADO))
        pares = tf.sin(posCAMBIADO / denominador)
        print("pares", pares)
        print("pares shape", tf.shape(pares))
        impares = tf.cos(posCAMBIADO / denominador)
        pos_encoding = tf.concat([pares, impares], axis=-1)
        print("pos_encoding sumando pares e impares", pos_encoding)
        print("pos_encoding sumando pares e impares", tf.shape(pos_encoding))
        PE = inputs+ pos_encoding
        print("pos_encoding sumando los inputs", PE)
        print("pos_encoding sumando los inputs", tf.shape(PE))
        return PE


 EMBEDDING

In [305]:
class SentenceEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_sequence_length, d_model, vocab_size):
        super(SentenceEmbedding, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, d_model)
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = tf.keras.layers.Dropout(rate=0.3)

    def call(self, x):
        print("input", x)
        x = self.embedding(x)
        print("after embedding", x)
        x *=  tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        print("after normalizing", x)
        x = self.position_encoder(x)
        print("after position encoder", x)
        x = self.dropout(x)
        print("after dropout", x)
        return x

FEED FORDWARD

In [306]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
          tf.keras.layers.Dense(hidden, activation='relu'), #PRIMERO LO AMPLIO AL NUMERO DE NEURONAS QUE ME INDIQUE MI HIPERPARAMETRO HIDDEN
          tf.keras.layers.Dropout(drop_prob),#LE APLICO DROPOUT PARA EL OVERFITTING
          tf.keras.layers.Dense(d_model)#Y LO VUELVO A REDUCIR AL NUMERO INICIAL DE NEURONAS
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)


    def call(self, x):
        x = self.add([x, self.seq(x)]) #LE AÑADO EL VALOR RESIDUAL PARA NO TENER VANISING GRADIENTS
        x = self.layer_norm(x) #Y LO NORMALIZO
        return x


ENCODER LAYER ( SE COMPONE DE UN MULTIHEAD ATTENTION CON MASK DE PADDING, UNA CAPA DE ADD Y NORMALIZATION Y UN FEEDFORDWARD (DENTRO DE ESTE ESTÁ YA IMPLEMENTADO EL ULTIMO ADD AND NORM)

In [307]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, hidden, num_heads, drop_prob):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout = tf.keras.layers.Dropout(drop_prob)
        self.ffn = FeedForward(d_model, hidden, drop_prob)
        self.add = tf.keras.layers.Add()

    def call(self, x, mask = None, name = "encoder_layer"):
        residual = tf.identity(x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("residual shape:", tf.shape(residual))
        print("residual:", residual)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        x = self.attention({"x": x, "mask": mask})#padding mask
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("DESPUES DE ATTENTION shape:", tf.shape(x))
        print("DESPUES DE ATTENTION:", x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        x = self.dropout(x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de dropout shape:", tf.shape(x))
        print("despues de dropout:", x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        x = self.add([x, residual])
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de add shape:", tf.shape(x))
        print("despues de add:", x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        x = self.norm(x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de norm shape:", tf.shape(x))
        print("despues de norm:", x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        x = self.ffn(x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de fnn shape:", tf.shape(x))
        print("despues de fnn:", x)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        return x

AHORA EL ECONDER ENTERO, TENGO QUE AÑADIR EL EMBEDDING, EL POSITIONAL ENCODER Y HACER LAS MULTICAPAS DE ENCODER

In [308]:
import tensorflow as tf

class Encoder(tf.keras.layers.Layer):
    def __init__(self, d_model, hidden, num_heads, drop_prob, num_layers, max_sequence_length, vocab_size):
        super(Encoder, self).__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, vocab_size)
        self.layers = [EncoderLayer(d_model, hidden, num_heads, drop_prob) for _ in range(num_layers)]
        self.num_layers = num_layers

    def call(self, x, mask):
        x = self.sentence_embedding(x)
        for i in range(self.num_layers):
            x = self.layers[i](x, mask)
            print("vueltas del encoder", tf.shape(x))
        return x



ENCODER

In [309]:
class EncDecAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
      super(EncDecAttention, self).__init__()
      self.d_model = d_model  #es la embedding dimension
      self.num_heads = num_heads
      self.head_dim = d_model // num_heads #cada cabeza tiene esta dimension
      self.kv_layer = tf.keras.layers.Dense(2 * d_model) #hacemos la dimension de emmbedding por tres para hacer en el mismo tensor q, k y v
      self.q_layer = tf.keras.layers.Dense(d_model)
      self.linear_layer = tf.keras.layers.Dense(d_model)

  def call(self, inputs):
        enc_outputs, dec_inputs, mask = (
            inputs["enc_outputs"],#x
            inputs["dec_inputs"],
            inputs["mask"]
        )
        print("enc_outputs", enc_outputs)
        print("dec_inputs", dec_inputs)
        print("mask", mask)
        #batch_size, sequence_length, d_model = tf.shape(enc_outputs) #entra como 4 (batch_size), 140 seq_lenght, 512 d_model
        batch_size, sequence_length, d_model = tf.compat.v1.Session().run(tf.compat.v1.shape(enc_outputs)).tolist()
        print("batch_size, sequence_length, d_model", batch_size, sequence_length, d_model)
        print("tf.shape(x)", tf.shape(enc_outputs))
        kv = self.kv_layer(enc_outputs) #lo pasamos de d_model a d_modelx3
        print("tf.shape(kv) por 2", tf.shape(kv))
        q = self.q_layer(dec_inputs)
        print("tf.shape(q) por 1", tf.shape(q))
        kv = tf.reshape(kv, (batch_size, sequence_length, self.num_heads, 2 * self.head_dim)) #lo dividimos en las cabezas
        print("tf.shape(kv) le cambiamos a 4 dim", tf.shape(kv))
        q = tf.reshape(q, (batch_size, sequence_length, self.num_heads,  self.head_dim)) #lo dividimos en las cabezas
        print("tf.shape(q) le cambiamos a 4 dim", tf.shape(q))
        kv = tf.transpose(kv, perm=(0, 2, 1, 3))#le cambiamod el orden para que esten las cabezas primero
        print("tf.shape(kv) cambiamos de orden la cabeza", tf.shape(kv))
        q = tf.transpose(q, perm=(0, 2, 1, 3))#le cambiamod el orden para que esten las cabezas primero
        print("tf.shape(q) cambiamos de orden la cabeza", tf.shape(q))
        k, v = tf.split(kv, num_or_size_splits=2, axis=-1)#lo dividimos en 2 por la ultima dimension
        print("kv", tf.shape(k),tf.shape(v))
        values = scaled_dot_product(q, k, v, mask) #lo pasamos por la funcion de atencion que nos devuelve un tensor de 4, 8, 140, 64
        print("values", tf.shape(values))#TENGO QUE METER EN LA LINEA DE ANTES MASKPADDING
        values = tf.reshape(tf.transpose(values, perm=(0, 2, 1, 3)), (batch_size, sequence_length, self.num_heads * self.head_dim)) #lo volvemos a poner primero seqlen y luego la dimension de las cabezas y las juntamos
        print("values cambiad0", tf.shape(values))
        out = self.linear_layer(values)
        print("out", tf.shape(out))
        print("out", out)
        return out



PROBANDO LA CROSS ATTENTION

DECODER LAYER

In [310]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, hidden, num_heads, drop_prob):
      super().__init__()
      self.attention1 = MultiHeadAttention(d_model, num_heads)
      self.norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
      self.dropout = tf.keras.layers.Dropout(drop_prob)

      self.attention2 = EncDecAttention( d_model, num_heads)
      self.norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
      self.dropout2 = tf.keras.layers.Dropout(drop_prob)
      self.ffn = FeedForward(d_model, hidden, drop_prob)
      self.add = tf.keras.layers.Add()
  def call(self, enc_outputs, dec_inputs, lookAheadMask = None, encDecMask = None):
        residual = tf.identity(dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("residual shape:", tf.shape(residual))
        print("residual:", residual)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        dec_inputs = self.attention1({"x": dec_inputs, "mask": lookAheadMask})
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("DESPUES DE ATTENTION shape:", tf.shape(dec_inputs))
        print("DESPUES DE ATTENTION:", dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        dec_inputs = self.dropout(dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de dropout shape:", tf.shape(dec_inputs))
        print("despues de dropout:", dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        dec_inputs = self.add([dec_inputs, residual])
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de add shape:", tf.shape(dec_inputs))
        print("despues de add:", dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        dec_inputs = self.norm(dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de norm shape:", tf.shape(dec_inputs))
        print("despues de norm:", dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        residual = tf.identity(dec_inputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("residual shape:", tf.shape(residual))
        print("residual:", residual)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        encDecOutputs = self.attention2({"enc_outputs": enc_outputs,"dec_inputs":dec_inputs,  "mask": encDecMask})
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("DESPUES DE ATTENTION 222222shape:", tf.shape(encDecOutputs))
        print("DESPUES DE ATTENTION:222", encDecOutputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        encDecOutputs = self.dropout2(encDecOutputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de dropout 222222 shape:", tf.shape(encDecOutputs))
        print("despues de dropout222222:", encDecOutputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        encDecOutputs = self.add([encDecOutputs, residual])
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de add 2222 shape:", tf.shape(encDecOutputs))
        print("despues de 222add:", encDecOutputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        encDecOutputs = self.norm2(encDecOutputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de norm 2222 shape:", tf.shape(encDecOutputs))
        print("despues de 222norm:", encDecOutputs)
        encDecOutputs = self.ffn(encDecOutputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        print("despues de fnn shape:", tf.shape(encDecOutputs))
        print("despues de fnn:", encDecOutputs)
        print("-------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------")
        return encDecOutputs


me queda el mask del cross attention


In [311]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self,d_model, hidden, num_heads, drop_prob, num_layers, max_sequence_length, vocab_size):
      super().__init__()
      self.embedding = SentenceEmbedding(max_sequence_length, d_model, vocab_size)
      self.layers = [DecoderLayer(d_model, hidden, num_heads, drop_prob) for _ in range(num_layers)]
      self.num_layers = num_layers

  def call(self,  enc_outputs, dec_inputs, lookAheadMask = None, encDecMask = None):
        dec_inputs = self.embedding(dec_inputs)
        for i in range(self.num_layers):
            dec_inputs = self.layers[i](enc_outputs, dec_inputs, lookAheadMask, encDecMask)
            print("vueltas del encoder", tf.shape(dec_inputs))
        return dec_inputs

otra cosa que puedo hacer es cambiar la mask de orden en enc mask

y otra peuba es usar la de google

y otra prueba es cambiar los 24.4555 a 0

la clase transformer

In [312]:
class Transformer(tf.keras.Model):
  def __init__(self,d_model, hidden, num_heads, drop_prob, num_layers, max_sequence_length, vocab_size):
      super().__init__()
      self.encoder = Encoder(d_model, hidden, num_heads, drop_prob, num_layers, max_sequence_length, vocab_size)
      self.decoder = Decoder(d_model, hidden, num_heads, drop_prob, num_layers, max_sequence_length, vocab_size)
      self.linear = tf.keras.layers.Dense(vocab_size)
  def call(self,  inputs):
      try:
            enc_inputs, dec_inputs = (
                inputs["enc_inputs"],
                inputs["dec_inputs"]
            )
            print(tf.shape(enc_inputs))
            print(enc_inputs, "ESTOS2")
            encMask = crear_masks(enc_inputs)
            print("encMask", encMask)
            lookmask = create_look_ahead_mask(dec_inputs)
            lookAheadMask = crear_masks(dec_inputs, lookmask)
            print("maskLookAhe", lookAheadMask)
            maskCross = create_mask_cross(enc_inputs, dec_inputs)
            print("maskCross", maskCross)
            x = self.encoder(enc_inputs, encMask)
            x = self.decoder(x, dec_inputs, lookAheadMask, maskCross)
            print(tf.shape(x))
            x = self.linear(x)
            print(tf.shape(x))
            print("FINAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAL", x)

      except Exception as e:
            print("An exception occurred in Transformer call:")
            print(str(e))
            # Puedes agregar más información de depuración aquí si es necesario
            raise e


##ENTRENAMIENTO

In [313]:
# For tf.data.Dataset
#BUFFER_SIZE = 20000


EPOCHS = 40

BATCH_SIZE = 2
SEQUENCE_LENGTH = 120
D_MODEL = 512
HIDDEN = 1024
DROP_PROB = 0.1
NUM_HEADS = 8
NUM_LAYERS = 5
vocab_size = tokenizerTotal.get_vocab_size()+2

print(vocab_size)

1031


### Loss function

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [314]:
def loss_function(y_true, y_pred):
    if y_true is None or y_pred is None:
        raise ValueError("y_true or y_pred is None")

    print("y_true shape:", y_true.shape)
    print("y_true type:", type(y_true))
    print("y_pred shape:", y_pred.shape)
    print("y_pred type:", type(y_pred))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=False, reduction="none"
    )(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)


### Custom learning rate

Use the Adam optimizer with a custom learning rate scheduler according to the formula in the [paper](https://arxiv.org/abs/1706.03762).

$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$

In [315]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.constant(d_model)
        self.warmup_steps = warmup_steps

    def get_config(self):
        return {"d_model": self.d_model, "warmup_steps": self.warmup_steps}

    def __call__(self, step):
        print("custom ")
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.multiply(
            tf.math.rsqrt(self.d_model), tf.math.minimum(arg1, arg2)
        )

### Initialize and compile model

Initialize and compile model with our predefined custom learning rate and Adam optimizer under the strategy scope.

In [316]:
# clear backend
tf.keras.backend.clear_session()

learning_rate = 0.001

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)


def accuracy(y_true, y_pred):
    #ensure labels have shape (batch_size, SEQUENCE_LENGTH - 1)
    y_true = tf.reshape(y_true, shape=(-1, SEQUENCE_LENGTH - 1))
    print("accurCY")
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


In [317]:
with strategy.scope():
    model = Transformer(D_MODEL, HIDDEN, NUM_HEADS, DROP_PROB, NUM_LAYERS, SEQUENCE_LENGTH, vocab_size)

    model.compile(optimizer=optimizer, loss=loss_function)



In [318]:
numpy_iterator = tf.data.Dataset.get_single_element(dataset.batch(len(dataset)))

# Convertir el iterador de TensorFlow a NumPy
dataset_values = list(tf.nest.map_structure(lambda x: x.numpy(), numpy_iterator))

# Extraer cada parte en variables separadas
inputs_tensor = tf.convert_to_tensor(numpy_iterator[0]['inputs'])
dec_inputs_tensor = tf.convert_to_tensor(numpy_iterator[0]['dec_inputs'])
outputs_tensor = tf.convert_to_tensor(numpy_iterator[1]['outputs'])

In [319]:
inputs_tensor

<tf.Tensor: shape=(9, 2, 120), dtype=int32, numpy=
array([[[  96,  168,  152, ...,   67,    4, 1030],
        [  96,  168,  152, ...,   67,    4, 1030]],

       [[  96,  168,  152, ...,   67,    4, 1030],
        [  96,  168,  152, ...,   67,    4, 1030]],

       [[ 607,   66,   37, ...,  687,    6, 1030],
        [ 607,   66,   37, ...,  687,    6, 1030]],

       ...,

       [[1029,  137,  391, ...,    0,    0,    0],
        [1029,  137,  391, ...,    0,    0,    0]],

       [[1029,  137,  391, ...,    0,    0,    0],
        [1029,  218, 1005, ...,    0,    0,    0]],

       [[1029,  285,  317, ...,    0,    0,    0],
        [1029,  285,  317, ...,    0,    0,    0]]], dtype=int32)>

In [320]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "enc_inputs": inputs_tensor,
        "dec_inputs": dec_inputs_tensor
    },
    outputs_tensor  # aquí colocas tus etiquetas
))


print(train_dataset)
# Después, puedes llamar a model.fit directamente
model.fit(train_dataset, epochs=EPOCHS)


<TensorSliceDataset element_spec=({'enc_inputs': TensorSpec(shape=(2, 120), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(2, 120), dtype=tf.int32, name=None)}, TensorSpec(shape=(2, 120), dtype=tf.int32, name=None))>
Epoch 1/40
Tensor("transformer/Shape:0", shape=(2,), dtype=int32)
Tensor("IteratorGetNext:1", shape=(2, 120), dtype=int32) ESTOS2
encMask Tensor("transformer/Maximum_1:0", shape=(2, 1, 120, 120), dtype=float32)
maskLookAhe Tensor("transformer/Maximum_3:0", shape=(2, 1, 120, 120), dtype=float32)
maskCross Tensor("transformer/Maximum_5:0", shape=(2, 1, 120, 120), dtype=float32)
input Tensor("IteratorGetNext:1", shape=(2, 120), dtype=int32)
after embedding Tensor("transformer/encoder/sentence_embedding/embedding/embedding_lookup/Identity_1:0", shape=(2, 120, 512), dtype=float32)
after normalizing Tensor("transformer/encoder/sentence_embedding/mul:0", shape=(2, 120, 512), dtype=float32)
i2 Tensor("transformer/encoder/sentence_embedding/positional_encoding/range:0"

AttributeError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/engine/compile_utils.py", line 199, in __call__
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "/usr/local/lib/p